In [106]:
import pandas as pd

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

2022-06-23 01:48:00.207817: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-23 01:48:00.207852: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import enum
import pandas as pd 
import os

import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
import pyarrow.parquet as pq
import pyarrow as pa
import glob

import sys
import os
sys.path.append("/scratch/venia/web2wiki/")
sys.setrecursionlimit(10000)

import re
import time
from bs4 import BeautifulSoup

from settings import WIKI_PAGES_DIR

os.environ['SPARK_HOME'] = "/home/veselovs/spark-3.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/veselovs/jdk-13.0.2"
spark = SparkSession.builder.getOrCreate()




22/06/23 01:48:24 WARN Utils: Your hostname, iccluster039 resolves to a loopback address: 127.0.1.1; using 10.90.38.15 instead (on interface ens786f0)
22/06/23 01:48:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/23 01:48:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/23 01:48:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
files = glob.glob(WIKI_PAGES_DIR+"/*")[0:400]


regex_search = r"(en.wikipedia\.org[\s\/a-zA-ZäöüÄÖÜßþóúí\_\?(\),\,\-\#\&\$\@\!0-9\.\%\–\'\:\!]+)"

@F.udf
def extract_regex(x):
    links = re.findall(regex_search, x)
    return links

def extract_soup(x: str, only_para = False):
    soup = BeautifulSoup(x,features="lxml")
    text = soup.get_text()
    
    return text


extract_soup_udf = F.udf(extract_soup)

In [11]:
df = spark.read.load(files)

# we will sample 10% of the entries
sampled = df.sample(0.1)
text = sampled.withColumn("text",extract_soup_udf(F.col("content")))
text = text.select("url","text").toPandas()

# we saved the file.

Define the embedding and save it 

In [21]:
txt= pd.read_parquet("/scratch/venia/web2wiki/data/url_text.parquet",index=False)
sentences = txt["text"].values

embeddings = model.encode(sentences, device = "cuda:1")

In [25]:
txt["url"].to_csv("/home/veselovs/html/urls.tsv", sep = "\t", index=False, header = None)
pd.DataFrame(embeddings).to_csv("/home/veselovs/html/vecs.tsv",sep="\t", index=False, header = None)

In [29]:
embedding = pd.DataFrame(embeddings)

In [82]:
embedding["url"] = txt["url"]

## Cluster the embedding and assign names
- Run KMeans on the embeddings
- Determine name holistically based on domain name


1. Cluster the embedding

In [31]:
dims = list(range(768))

In [129]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

def run_kmeans(input_col, n_clusters = 30):
    km = KMeans(n_clusters)
    scores = km.fit_predict(input_col)
    return scores

def find_centroid(df, cluster):
    temp = df[df["cluster"] == cluster]
    urls = temp["url"]
    temp = temp[dims].values
    centroid = temp.mean(0).reshape(1,-1)
    cosine_sim = cosine_similarity(temp,centroid).reshape(-1)
    return cosine_sim, urls

def centroid_sim_dataframes(cosine_sim):
    df = pd.DataFrame(cosine_sim).T.rename({0: "centroid_sim",1: "url"},axis=1)
    df = df[["url","centroid_sim"]]
    df = df.sort_values(by = "centroid_sim",ascending=False)
    return df


In [ ]:
clusters = run_kmeans(embedding[dims])
embedding["cluster"] = clusters

In [140]:
embedding.to_csv("/scratch/venia/web2wiki/data/url_embedding.csv", index=False)

2. Name the embedding

In [135]:
cosine_sims = {}
for cluster in range(30):
    cosine_sims[cluster] = centroid_sim_dataframes(find_centroid(embedding, cluster))

In [141]:
for cluster in range(30):
    cosine_sims[cluster].to_csv(f"/scratch/venia/web2wiki/embedding_generation/bert/data/{cluster}_centroid_sim.csv",index=False)

In [158]:
for cluster in range(30):
    print(cluster,cosine_sims[cluster]["centroid_sim"].mean())

0 0.6182853310370484
1 0.913824386755295
2 0.7699986529069053
3 0.737997625291962
4 0.7154784091683322
5 0.4700611638933027
6 0.9455930269681491
7 0.5880411255049803
8 0.483544480065558
9 0.6091528130002436
10 0.6472557354183913
11 0.59940131356783
12 0.6096128592291382
13 0.6148600955661252
14 0.6327055514215709
15 0.7666011817102552
16 0.8793999354044596
17 0.6910316438923095
18 0.6277375060242492
19 0.4773903375939478
20 0.5658946697528545
21 0.5216715166645666
22 0.7474893318531652
23 0.5914238207267992
24 0.7118936063941389
25 0.5402168273925781
26 0.9355182960385182
27 0.6104378736656131
28 0.6981915521888094
29 0.7762993633360894


In [161]:
cosine_sims[12]

,url,centroid_sim
398,http://www.atomicmall.com/view?id=Australia-Ma...,0.823941
238,http://www.atomicmall.com/view?id=Stretch-Lace...,0.823873
523,http://www.atomicmall.com/view?id=For-Galaxy-N...,0.817096
533,http://www.atomicmall.com/view?id=Digital-Medi...,0.805082
122,http://www.atomicmall.com/related.php?pouch=Q0...,0.803883
...,...,...
320,http://www.seamheads.com/NegroLgs/manager.php?...,0.43161
195,http://seamheads.com/NegroLgs/player.php?playe...,0.43161
436,http://mail.seamheads.com/NegroLgs/player.php?...,0.431609
393,http://www.tattooexpo09.com/will-cbd-fail-a-ha...,0.415856


(16804,)